In [1]:
import ocsmesh
import geopandas as gpd

In [7]:
## Inputs:
path = r"C:\Users\Felicio.Cassalho\Work/Modeling/NHC/"
points = gpd.read_file(path+"/sample.shp")
stofs_mesh = ocsmesh.Mesh.open(path+"/stofs_mesh.2dm", crs=4326).msh_t

In [3]:
# Here you control the size of the area to be re-meshed
clip_gdf = gpd.GeoDataFrame(geometry =
                              gpd.GeoSeries(points.union_all().buffer(0.005)),
                                crs=4326).explode()
# clip_gdf.to_file(path+"/clip_gdf.shp")

In [4]:
# Creating the mesh for the area above (if you want to change the mesh resolution, change the decimal value below):
clip_mesh = ocsmesh.utils.triangulate_polygon(clip_gdf,opts='pq30a.0000002')
# ocsmesh.Mesh(clip_mesh).write(path+"/clip_mesh.2dm", format='2dm', overwrite=True)

In [ ]:
## Carve the STOFS mesh with the clip_mesh. Here buffer size and adjacent layers control how much you will erode from the STOFS mesh
carved_mesh = ocsmesh.utils.clip_mesh_by_mesh(stofs_mesh,
                                              clip_mesh,
                                              adjacent_layers=2,
                                              buffer_size=0.0075
                                              )
# ocsmesh.Mesh(carved_mesh).write(path+"/carved_mesh.2dm", format='2dm', overwrite=True)

In [ ]:
## This is a mesh between carved_mesh and clip_mesh
buff_mesh = ocsmesh.utils.create_mesh_from_mesh_diff([stofs_mesh,clip_mesh],
                                                     carved_mesh,
                                                     clip_mesh,
                                                     min_int_ang=25,
                                                     buffer_domain=0.01,
                                                     )
# ocsmesh.Mesh(buff_mesh).write(path+"/buff_mesh.2dm", format='2dm', overwrite=True)

In [ ]:
## Now we just combined all the 3 meshes:
msht_combined = ocsmesh.utils.merge_neighboring_meshes(buff_mesh,
                                                       carved_mesh,
                                                       clip_mesh
                                                       )

In [ ]:
## Final mesh cleanup:
ocsmesh.utils.cleanup_duplicates(msht_combined)
ocsmesh.utils.cleanup_isolates(msht_combined)
ocsmesh.utils.put_id_tags(msht_combined)

In [ ]:
ocsmesh.Mesh(msht_combined).write(path+"/final_mesh.2dm", format='2dm', overwrite=True)